In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import keras
from sklearn.preprocessing import MinMaxScaler


2024-11-06 11:19:21.007419: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-06 11:19:21.018067: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-06 11:19:21.028360: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-06 11:19:21.136495: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730891961.174691    5837 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730891961.18

In [ ]:
file_path = 'https://docs.google.com/spreadsheets/d/1V6cBNRohsvWOEcPUMWyZ6tMAvc25Ji2U/export?format=csv'
data = pd.read_csv(file_path)


In [ ]:

# Ensure all values are numeric (convert non-numeric values to NaN)
data = data.apply(pd.to_numeric, errors='coerce').fillna(0)
# Remove rows where all values are zero
data = data.loc[~(data.eq(0).all(axis=1))]

data['resi_value'] = data.apply(lambda row : row['resi_value'] if row['trend_type']==-1 else 100-row['resi_value'], axis = 1)

data.tail(30)



In [ ]:
# # replace all negative values in max_price_move with 0 because out stoploss has been hitted anyway
# data["max_price_move"] = data["max_price_move"].apply(lambda x: max(0, x))
# data

In [ ]:
reserve = data
data = data.drop(['time','price','trend_type','max_seen', 'min_seen'], axis=1)
# data = data.drop(['rsi_filter','resistance_filter'], axis=1)

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.shape

In [ ]:
data.isnull().sum()

In [ ]:
# normalized_df = keras.utils.normalize(data)
# type(normalized_df)
# data = pd.DataFrame(normalized_df, columns=["leg_3_to_leg_1_time","leg_2_to_leg_1_time","leg_2_correction_percent","resistance","rsi_filter_passed","normalized_rsi","max_price_move"])
# data

In [ ]:
#normalize data
columns_to_normalize = data.columns.difference(['rsi_filter'])
scaler = MinMaxScaler()
data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])

#print(data)



In [ ]:
import neat
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Example data setup, replace with your actual dataset
data = pd.DataFrame({
    'leg_3_to_1': np.random.randn(100),
    'leg_2_to_1': np.random.randn(100),
    'leg_2_correction': np.random.randn(100),
    'resistance_filter': np.random.randn(100),
    'rsi_filter': np.random.randn(100),
    'resi_value': np.random.randn(100),
    'max_move': np.random.randn(100)
})

# Separate features and target variable
X = data[['leg_3_to_1', 'leg_2_to_1', 'leg_2_correction', 'resistance_filter', 'rsi_filter', 'resi_value']]
y = data['max_move']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the NEAT fitness function
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        # Convert NEAT genome to TensorFlow model
        model = create_tf_model_from_genome(genome, config, input_shape=X_train_scaled.shape[1])

        # Compile the model
        model.compile(optimizer='adam', loss='mse')

        # Train the model
        model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, verbose=0)

        # Evaluate and set the fitness score
        predictions = model.predict(X_test_scaled)
        mse = np.mean((predictions.flatten() - y_test) ** 2)
        genome.fitness = -mse  # Using negative MSE to maximize fitness

# Function to create a TensorFlow model from a NEAT genome
def create_tf_model_from_genome(genome, config, input_shape):
    model = tf.keras.Sequential()
    
    # Input layer
    model.add(tf.keras.layers.InputLayer(input_shape=(input_shape,)))

    # Hidden layers based on genome connections
    for node_key, node in genome.nodes.items():
        if node_key not in config.genome_config.output_keys:
            activation = 'relu' if node.bias > 0 else 'sigmoid'
            model.add(tf.keras.layers.Dense(units=10, activation=activation))
    
    # Output layer
    model.add(tf.keras.layers.Dense(1))  # Regression output

    return model

# Load NEAT configuration
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     'config-feedforward')

# Initialize population and evolution reporters
population = neat.Population(config)
population.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
population.add_reporter(stats)

# Run NEAT
winner = population.run(eval_genomes, n=20)

# Evaluate the best evolved model
best_model = create_tf_model_from_genome(winner, config, input_shape=X_train_scaled.shape[1])
best_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
best_model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, verbose=1)
test_loss, test_mae = best_model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"Test MAE of best evolved model: {test_mae}")


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.figure(figsize=(12, 5))

# Loss
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()

# Mean Absolute Error
plt.subplot(1, 2, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model Mean Absolute Error')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()

plt.show()


In [ ]:
test_loss, test_mae = model.evaluate(X_test_scaled, y_test, verbose=1)
print(f"Test Loss (MSE): {test_loss}")
print(f"Test Mean Absolute Error (MAE): {test_mae}")


In [ ]:
import numpy as np

# Make predictions
predictions = model.predict(X_test_scaled).flatten()

# Compare predictions with actual values
comparison_df = pd.DataFrame({'Actual': y_test, 'Predicted': predictions})
print(comparison_df.head(10))  # Display first 10 rows for a quick check
